# Shunting Yard Calculator
## Introduction

This notebook serves to more intuitively explain the code in ```main.py```, The initial python script I wrote for this exercise. 

The program queries the user for a math expression to evaluate. It supports five operators: +, -, *, /, and *.

The diagram below explains the purpose of each function:
![Alt text](./mermaid-diagram-2023-09-26-223147.png)

- The ```tokenizer``` takes the user input string and converts it to a list of tokens. A token is either a number or an operator.
- The ```shunting_yard``` algorithm converts infix notation to postfix notation (reverse polish notation). For example, [$1 + 1$] becomes [$1$ $1$ $+$]. This removes the need for parenthesis.

In [4]:
# TODO: account for floating point numbers and negative numbers
def tokenizer(user_input: str, operator_list: list) -> list:
    token_builder = ""
    tokenized_list = []
    for char in user_input:
        if char.isdigit():
            token_builder += char
        elif char in operator_list:
            tokenized_list.append(token_builder)
            tokenized_list.append(char)
            token_builder = ""
        else:
            continue
    tokenized_list.append(token_builder)
    return tokenized_list

TypeError: tokenizer() missing 1 required positional argument: 'operator_list'

In [2]:
def shunting_yard(tokenized_expression: list, operator_precedence: dict):
    output = []
    operator_stack = []

    opening_parenthesis_counter = 0
    closing_parenthesis_counter = 0

    for token in tokenized_expression:
        if token.isdigit():
            output.append(token)
        elif token == '(':
            opening_parenthesis_counter += 1
            operator_stack.append(token)
        elif token == ')':
            closing_parenthesis_counter += 1
            while operator_stack and operator_stack[-1] != "(":
                output.append(operator_stack.pop())
            operator_stack.pop()
        elif token in operator_precedence.keys():
            while operator_stack and operator_precedence[token] <= operator_precedence[operator_stack[-1]] and operator_stack[-1] != '(':
                output.append(operator_stack.pop())
            operator_stack.append(token)
        else:
            continue

    while operator_stack:
        output.append(operator_stack.pop())

    if opening_parenthesis_counter != closing_parenthesis_counter:
        print("warning: parenthesis mismatch detected")

    return output

In [3]:
def rpn_evaluate(expression: list):
    for token in expression[:]:
        if not (token.isdigit()):
            token_index = expression.index(token)

            if token == '+':
                expression[token_index-2] = float(expression[token_index-2]) + float(expression[token_index-1])
            if token == '-':
                expression[token_index-2] = float(expression[token_index-2]) - float(expression[token_index-1])
            if token == '*':
                expression[token_index - 2] = float(expression[token_index - 2]) * float(expression[token_index - 1])
            if token == '/':
                if float(expression[token_index - 1]) == 0:
                    print("[ERROR] ignoring division by zero, result will be incorrect")
                    print("RPN: " + str(expression))
                    continue
                expression[token_index - 2] = float(expression[token_index - 2]) / float(expression[token_index - 1])
            if token == '^':
                expression[token_index - 2] = float(expression[token_index - 2]) ** float(expression[token_index - 1])

            expression.pop(token_index)
            expression.pop(token_index-1)

    return expression[0]